#### Importo librerías

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, roc_curve
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

#### Limpieza de datos para el modelo

In [2]:
data = pd.read_csv("C:/Users/paula/OneDrive/Documentos/Facultad/Análisis Predictivo/final_predictivo/dataset.csv")
data = data.drop(['Unnamed: 0'], axis=1)
data = data.drop(['track_id'], axis=1)
data = data.drop(['album_name'], axis=1)
data = data.drop(['track_name'], axis=1)

# Pruebo escalar los datos numéricos
scaler = StandardScaler()
numeric_features = ['duration_ms', 'danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
data[numeric_features] = scaler.fit_transform(data[numeric_features])

#### Funcion para calcular las estadisticas de las distintas características de las canciones por género

In [3]:
def agregar_estadisticas_por_genero(data, columna, genero):

    estadisticas_por_genero = data.groupby(genero)[columna].agg(['max', 'min', 'median', 'mean']).reset_index()

    data = pd.merge(data, estadisticas_por_genero.rename(columns={'max': columna + '_max_X_' + genero,
                                                                 'min': columna + '_min_X_' + genero,
                                                                 'median': columna + '_median_X_' + genero,
                                                                 'mean': columna + '_mean_' + genero}),
                    on=genero,
                    how='left')

    return data

data = agregar_estadisticas_por_genero(data, 'energy', 'track_genre')
data = agregar_estadisticas_por_genero(data, 'danceability', 'track_genre')
data = agregar_estadisticas_por_genero(data, 'instrumentalness', 'track_genre')
data = agregar_estadisticas_por_genero(data, 'speechiness', 'track_genre')
data = agregar_estadisticas_por_genero(data, 'acousticness', 'track_genre')

#### Calculo las estadisticas para la duración de las canciones por artista

In [4]:
data['max_duration_by_artist'] = data.groupby('artists')['duration_ms'].transform('max')
data['min_duration_by_artist'] = data.groupby('artists')['duration_ms'].transform('min')
data['median_duration_by_artist'] = data.groupby('artists')['duration_ms'].transform('median')
mean_duration_all_artists = data['duration_ms'].mean()

# Imputo datos faltantes con los promedios de cada categoría
data['max_duration_by_artist'].fillna(mean_duration_all_artists, inplace=True)
data['min_duration_by_artist'].fillna(mean_duration_all_artists, inplace=True)
data['median_duration_by_artist'].fillna(mean_duration_all_artists, inplace=True)

In [5]:
# Veo cuales son los NAs pero no hago nada al respecto

print(data.isna().any().any()) 
print(data.isna().any())

True
artists                                   True
popularity                               False
duration_ms                              False
explicit                                 False
danceability                             False
energy                                   False
key                                      False
loudness                                 False
mode                                     False
speechiness                              False
acousticness                             False
instrumentalness                         False
liveness                                 False
valence                                  False
tempo                                    False
time_signature                           False
track_genre                              False
energy_max_X_track_genre                 False
energy_min_X_track_genre                 False
energy_median_X_track_genre              False
energy_mean_track_genre                  False
danceabi

#### Calculo la cantidad de explicits por artista y por género

In [6]:
data['explicit'] = data['explicit'].astype(int)
data['promedio_explicit_por_artista'] = data.groupby('artists')['explicit'].transform('mean')
mean_explicit_all_artists = data['explicit'].mean()
data['promedio_explicit_por_artista'].fillna(mean_explicit_all_artists, inplace=True)
data['promedio_explicit_por_genero'] = data.groupby('track_genre')['explicit'].transform('mean')
mean_explicit_all_artists = data['explicit'].mean()
data['promedio_explicit_por_genero'].fillna(mean_explicit_all_artists, inplace=True)

#### Hago label encoding para los artistas y generos (porque tienen que ser únicos)

In [7]:
# Etiquetado de artistas
artist_encoder = LabelEncoder()
data['artist_encoded'] = artist_encoder.fit_transform(data['artists'])
data = data.drop(['artists'], axis=1)
# Etiquetado de géneros de canciones
genre_encoder = LabelEncoder()
data['genre_encoded'] = genre_encoder.fit_transform(data['track_genre'])
data = data.drop(['track_genre'], axis=1)

#### Defino las variables a predecir

In [8]:
X = data.drop('explicit', axis=1)
y = data['explicit']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

#### Modelo Baseline - Regresión Logística

In [59]:
# Creo el modelo
modelo_regresion_logistica = LogisticRegression()

modelo_regresion_logistica.fit(X_train, y_train) # Ajusto el modelo a mis datos de entrenamiento (X_train, y_train)

predicciones = modelo_regresion_logistica.predict(X_test) # Hago predicciones con el conjunto de prueba (X_test)

exactitud = accuracy_score(y_test, predicciones)
print("Exactitud:", exactitud)

precision = precision_score(y_test, predicciones) # Calculo la precisión
print("Precisión:", precision)

recall = recall_score(y_test, predicciones) # Calculo el recall
print("Recall:", recall)

# Calcular el área bajo la curva ROC (AUC-ROC)
# Probabilidades de predicción para la clase positiva
probabilidades_positivas = modelo_regresion_logistica.predict_proba(X_test)[:, 1]
auc_roc = roc_auc_score(y_test, probabilidades_positivas)
print("AUC-ROC:", auc_roc)


Exactitud: 0.9253070175438597
Precisión: 0.621160409556314
Recall: 0.2849686847599165
AUC-ROC: 0.8803395153425284


C:\Users\paula\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


#### XG BOOST

In [13]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

dtrain = xgb.DMatrix(X_train, label=y_train) # Defino el objeto DMatrix para los datos de entrenamiento

#param_grid = { # Defino los parámetros del modelo XGBoost para clasificación
#    'max_depth': [3, 6, 9],
#    'min_child_weight': [1, 5, 10],
#    'subsample': [0.7, 0.8, 0.9],
#    'colsample_bytree': [0.7, 0.8, 0.9],
#    'learning_rate': [0.01, 0.1, 0.2],
#    'n_estimators': [100, 200, 300]
#}
#xgb_model = xgb.XGBClassifier(objective='multi:softmax', num_class=len(y.unique())) # Inicializo el modelo XGBoost
#grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1) # Inicializo GridSearchCV
#grid_search.fit(X_train, y_train) #Busco los mejores hiperparametros

best_params = {'colsample_bytree': 0.9, 'learning_rate': 0.1, 'max_depth': 9, 'min_child_weight': 1, 'n_estimators': 300, 'subsample': 0.8}
print("Mejores hiperparámetros:", best_params)

best_xgb_model = xgb.XGBClassifier(**best_params)# Entreno el modelo con los mejores hiperparámetros
best_xgb_model.fit(X_train, y_train)

predictions = best_xgb_model.predict(X_test) # Hago predicciones en el conjunto de prueba

exactitud = accuracy_score(y_test, predicciones)
print("Exactitud:", exactitud)

precision = precision_score(y_test, predicciones) # Calculo la precisión
print("Precisión:", precision)

recall = recall_score(y_test, predicciones) # Calculo el recall
print("Recall:", recall)

# Calculo el área bajo la curva ROC (AUC-ROC)
probabilidades_positivas = best_xgb_model.predict_proba(X_test)[:, 1]
auc_roc = roc_auc_score(y_test, probabilidades_positivas)
print("AUC-ROC:", auc_roc)

Mejores hiperparámetros: {'colsample_bytree': 0.9, 'learning_rate': 0.1, 'max_depth': 9, 'min_child_weight': 1, 'n_estimators': 300, 'subsample': 0.8}
Exactitud: 0.9742105263157895
Precisión: 0.8966547192353644
Recall: 0.7834029227557411
AUC-ROC: 0.995933397284693


#### Decision Tree

In [14]:
# Defino el modelo de árbol de decisión
decision_tree = DecisionTreeClassifier()

# Defino los rangos de los hiperparámetros a explorar
#param_dist = {
#    'max_depth': [3, 5, 7, 9, None],
#    'min_samples_split': [2, 5, 10],
#    'min_samples_leaf': [1, 2, 4],
#    'criterion': ['gini', 'entropy']
#}
#random_search = RandomizedSearchCV(decision_tree, param_distributions=param_dist, cv=5) # Realizo la búsqueda aleatoria de hiperparámetros con validación cruzada
#random_search.fit(X_train, y_train) # Entreno el modelo con la búsqueda aleatoria de hiperparámetros

best_params = {'min_samples_split': 10, 'min_samples_leaf': 1, 'max_depth': 7, 'criterion': 'gini'}
print("Mejores hiperparámetros:", best_params)

# Defino el modelo de árbol de decisión con los mejores hiperparámetros
best_decision_tree = DecisionTreeClassifier(**best_params)
best_decision_tree.fit(X_train, y_train)
predicciones = best_decision_tree.predict(X_test)

exactitud = accuracy_score(y_test, predicciones)
print("Exactitud:", exactitud)

precision = precision_score(y_test, predicciones)
print("Precisión:", precision)

recall = recall_score(y_test, predicciones)
print("Recall:", recall)

# Calculo el área bajo la curva ROC (AUC-ROC)
probabilidades_positivas = best_decision_tree.predict_proba(X_test)[:, 1]
auc_roc = roc_auc_score(y_test, probabilidades_positivas)
print("AUC-ROC:", auc_roc)

Mejores hiperparámetros: {'min_samples_split': 10, 'min_samples_leaf': 1, 'max_depth': 7, 'criterion': 'gini'}
Exactitud: 0.9741666666666666
Precisión: 0.8961194029850746
Recall: 0.7834029227557411
AUC-ROC: 0.9898691184708934


#### Random Forest

In [16]:
# Defino el modelo de Random Forest con hiperparámetros sugeridos
random_forest = RandomForestClassifier(n_estimators=2000, max_depth=None, min_samples_split=2, min_samples_leaf=1, random_state=42)

# Entreno el modelo
random_forest.fit(X_train, y_train)

# Hago predicciones en el conjunto de prueba
predictions = random_forest.predict(X_test)

exactitud = accuracy_score(y_test, predicciones)
print("Exactitud:", exactitud)

precision = precision_score(y_test, predicciones) # Calculo la precisión
print("Precisión:", precision)

recall = recall_score(y_test, predicciones) # Calculo el recall
print("Recall:", recall)

# Calculo el área bajo la curva ROC (AUC-ROC)
probabilidades_positivas = random_forest.predict_proba(X_test)[:, 1]
auc_roc = roc_auc_score(y_test, probabilidades_positivas)
print("AUC-ROC:", auc_roc)

Exactitud: 0.9741666666666666
Precisión: 0.8961194029850746
Recall: 0.7834029227557411
AUC-ROC: 0.9936978279263246
